# U.S. Medical Insurance Costs

In this project, we will be analyzing a CSV file with medical insurance data using Python. Our objective is to analyze the data to find summary statistics and what factors have the largest impacts on insurance cost.

In [1]:
# import libraries
import csv
# read csv file into all_data, a list of dictionaries of people
with open('insurance.csv') as insurance_csv:
    data = csv.DictReader(insurance_csv)
    all_data = []
    for row in data:
        all_data.append(row)
        
print('There are {} people in our data set.'.format(len(all_data)))
print('Our data set contains the following information about each person:')
print(all_data[0].keys())

There are 1338 people in our data set.
Our data set contains the following information about each person:
dict_keys(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'])


First we import the libraries we will be using to examine our data. Then we load the data into a list of dictionaries that represent a person. The keys of this dictionary are the columns from our CSV file. These are the columns and the possible values they may contain.

- age - the age of the person in years, an integer
- sex - the sex of the person, either 'male' or 'female'
- bmi - the body mass index of the person, a float
- children - the number of children the person has, an integer
- smoker - the smoking status of the person, either 'yes' or 'no'
- region - the region the where the person lives, either 'northeast', 'northwest', 'southeast', or 'southwest'
- charges - the amount of insurance charges the person paid that year, a float

Our data contains information on 1338 people. There are no missing values and the data is clean. All our data is stored as strings so we need to convert to the proper type to do any calculations.
We will add one more column to our data called 'has_children' which will contain the value 'yes' if the person has children and 'no' if the person does not.

In [2]:
for person in all_data:
    if person['children'] == '0':
        person['has_children'] = 'no'
    else:
        person['has_children'] = 'yes'

We need some helper functions to examine the data. These functions will analyze the data for summary statistics, and display those statistics for us.

In [3]:

# Function that finds the mean of a column in the data rounded to 2 decimal places
def find_mean_of_column(data, column):
    column_sum = 0
    for row in data:
        column_sum += float(row[column])
    return round(column_sum / len(data), 2)

# Function that finds the maximum value of a column in the data rounded to 2 decimal places
def find_max_of_column(data, column):
    column_max = float(data[0][column])
    for datum in data:
        if float(datum[column]) > column_max:
            column_max = float(datum[column])
    return round(column_max, 2)

# Function that finds the minimum value of a column in the data rounded to 2 decimal places
def find_min_of_column(data, column):
    column_min = float(data[0][column])
    for datum in data:
        if float(datum[column]) < column_min:
            column_min = float(datum[column])
    return round(column_min, 2)

# Function that finds the standard deviation of a column in the data rounded to 2 decimal places
def find_std_of_column(data, column):
    column_square_error_sum = 0
    column_mean = find_mean_of_column(data, column)
    for datum in data:
        column_square_error_sum += (float(datum[column]) - column_mean) ** 2
    return round((column_square_error_sum / (len(data) - 1))**.5, 2)

# Function that finds the mode of the column
def column_mode(data, column):
    counter = {}
    for datum in data:
        if datum[column] in counter:
            counter[datum[column]] += 1
        else:
            counter[datum[column]] = 1
    max_datum = data[0][column]
    max_count = counter[max_datum]
    for item in counter:
        if counter[item] > max_count:
            max_datum = item
            max_count = counter[item]
    return max_datum

# Function that returns a list of floats from column data
def make_column_float_list(data, column):
    column_list = []
    for datum in data:
        column_list.append(float(datum[column]))
    return column_list

# Function that finds the median of a column in the data rounded to 2 decimal places
def find_median_of_column(data, column):
    column_list = make_column_float_list(data, column)
    column_list.sort()
    length = len(column_list)
    if length % 2 == 1:
        mid_index = int((length - 1) / 2)
        return round(column_list[mid_index], 2)
    else:
        mid_index = int(length / 2)
        return round(((column_list[mid_index] + column_list[mid_index - 1]) / 2), 2)
    
# Function that finds the lower quartile of a column in data rounded to 2 decimal places   
def find_lower_quartile_of_column(data, column):
    column_list = make_column_float_list(data, column)
    column_list.sort()
    length = len(column_list)
    if length % 2 == 0:
        if int(length / 2) % 2 == 1:
            lower_index = int((length / 2 - 1) / 2)
            return round(column_list[lower_index], 2)
        else:
            lower_index = int(length / 4)
            return round(((column_list[lower_index] + column_list[lower_index - 1]) / 2), 2)
    else:
        if int((length - 1) / 2) % 2 == 1:
            lower_index = int((((length - 1) / 2) - 1) / 2)
            return round(column_list[lower_index], 2)
        else:
            lower_index = int((length - 1) / 4)
            return round(((column_list[lower_index] + column_list[lower_index - 1]) / 2), 2)

# Function that finds the upper quartile of a column in data rounded to 2 decimal places          
def find_upper_quartile_of_column(data, column):
    column_list = make_column_float_list(data, column)
    column_list.sort()
    length = len(column_list)
    if length % 2 == 0:
        if int(length / 2) % 2 == 1:
            lower_index = int((length / 2 - 1) / 2 + (length / 2))
            return round(column_list[lower_index], 2)
        else:
            lower_index = int(length / 4 + length / 2)
            return round(((column_list[lower_index] + column_list[lower_index - 1]) / 2), 2)
    else:
        if int((length - 1) / 2) % 2 == 1:
            lower_index = int((((length - 1) / 2) - 1) / 2 + (length + 1) / 2)
            return round(column_list[lower_index], 2)
        else:
            lower_index = int(3 * (length - 1) / 4)
            return round(((column_list[lower_index] + column_list[lower_index + 1]) / 2), 2)
    

# Function that creates a dictionary that groups people by the specified attribute
# as the key and the value of each key is a list of people (dictionaries) in that group
# this will be used to examine the summary statistics of various categories
def group_by_column(data, column):
    new_dict = {}
    for row in data:
        if row[column] in new_dict:
            new_dict[row[column]].append(row)
        else:
            new_dict[row[column]] = [row]
    sorted_new_dict = dict(sorted(new_dict.items()))
    return sorted_new_dict

# Function that creates a dictionary that groups people by the specified attribute
# as the key and the value of each key is the number of people in that group
def make_frequency_table(data, column):
    new_dict = group_by_column(data, column)
    dict_counter = {}
    for key in new_dict:
        dict_counter[key] = len(new_dict[key])
    sorted_dict_counter = dict(sorted(dict_counter.items()))
    return sorted_dict_counter

# Function that creates a dictionary that groups people by the specified attribute
# as the key and the value of each key is the proportion of people in that group to the 
# total number of people in the data set
def make_proportion_table(data, column):
    new_dict = group_by_column(data, column)
    dict_counter = {}
    for key in new_dict:
        dict_counter[key] = str(round(100 * float(len(new_dict[key]) / len(data)), 2)) + '%'
    sorted_dict_counter = dict(sorted(dict_counter.items()))
    return sorted_dict_counter   

# Function that creates a dictionary that groups people by the specified attribute
# as the key and the value of each key is the sum of the specified attribute in that group
def dict_key_value_sums(data, column):
    new_dict = group_by_column(data, column)
    dict_sums = {}
    for key in new_dict:
        dict_sums[key] = 0
        for index in new_dict[key]:
            dict_sums[key] += round(float(new_dict[key][index]), 2)
    return dict_sums

# Function for displaying numerical summary statistics including min, lower quartile,
# median, upper quartile, max, range, IQR, mean, and standard deviation
def display_numerical(data, column):
    column_mean = find_mean_of_column(data, column)
    column_max = find_max_of_column(data, column)
    column_min = find_min_of_column(data, column)
    column_std = find_std_of_column(data, column)
    column_median = find_median_of_column(data, column)
    column_lower_quartile = find_lower_quartile_of_column(data, column)
    column_upper_quartile = find_upper_quartile_of_column(data, column)
    print('          {}:'.format(column))
    print()
    print('     Min = {}'.format(column_min))
    print('     Lower Quartile = {}'.format(column_lower_quartile))
    print('     Median = {}'.format(column_median))
    print('     Upper Quartile = {}'.format(column_upper_quartile))
    print('     Max = {}'.format(column_max))
    print('     Range = {}'.format(round(column_max - column_min, 2)))
    print('     Interquartile Range = {}'.format(round(column_upper_quartile - column_lower_quartile, 2)))
    print('     Mean = {}'.format(column_mean))
    print('     Standard Deviation = {}'.format(column_std))
   
    
    print()
    
# Function for displaying categorical summary statistics including mode, a frequency table, and a
# proportion table
def display_categorical(data, column):
    frequency_table = make_frequency_table(data, column)
    proportion_table = make_proportion_table(data, column)
    print('          {}:'.format(column))
    print()
    print('     Mode = {}'.format(column_mode(data, column)))
    print('     Frequency = {}'.format(frequency_table))
    print('     Proportions = {}'.format(proportion_table))
    print()
    
    
# Function for displaying grouped numerical statistics
def display_group_stats(data, group, column):
    print('{}'.format(group.upper()))
    print()
    group_data = group_by_column(data, group)
    for key in group_data:
        print('     -{}-'.format(key))
        print()
        display_numerical(group_data[key], column)

        
    
# Function for summing variable 2 in groups partitioned by 1
# We never use this function
def group_by1_sum_by2(data, string1, string2):
    new_dict = group_by_column(data,string1)
    dict_sums = {}
    for key in new_dict:
        dict_sums[key] = 0
        for item in new_dict[key]:
            dict_sums[key] += float(item[string2])
        dict_sums[key] = round(dict_sums[key], 2)
    return dict_sums


This data is about insurance costs so we will examine that first. Below is the summary statistics for charges.

In [80]:
display_numerical(all_data, 'charges')

          charges:

     Min = 1121.87
     Lower Quartile = 4738.27
     Median = 9382.03
     Upper Quartile = 16657.72
     Max = 63770.43
     Range = 62648.56
     Interquartile Range = 11919.45
     Mean = 13270.42
     Standard Deviation = 12110.01



We can see that the mean is almost 50% more than the median which means either the distribution is skewed or there are outliers. Further analysis is needed.
We will analyze each other factor for summary statistics, whether there is an imbalance in representation, and how it affects insurance cost.
Next we will examine age.

In [81]:
display_numerical(all_data, 'age')
display_categorical(all_data, 'age')

          age:

     Min = 18.0
     Lower Quartile = 27.0
     Median = 39.0
     Upper Quartile = 51.0
     Max = 64.0
     Range = 46.0
     Interquartile Range = 24.0
     Mean = 39.21
     Standard Deviation = 14.05

          age:

     Mode = 18
     Frequency = {'18': 69, '19': 68, '20': 29, '21': 28, '22': 28, '23': 28, '24': 28, '25': 28, '26': 28, '27': 28, '28': 28, '29': 27, '30': 27, '31': 27, '32': 26, '33': 26, '34': 26, '35': 25, '36': 25, '37': 25, '38': 25, '39': 25, '40': 27, '41': 27, '42': 27, '43': 27, '44': 27, '45': 29, '46': 29, '47': 29, '48': 29, '49': 28, '50': 29, '51': 29, '52': 29, '53': 28, '54': 28, '55': 26, '56': 26, '57': 26, '58': 25, '59': 25, '60': 23, '61': 23, '62': 23, '63': 23, '64': 22}
     Proportions = {'18': '5.16%', '19': '5.08%', '20': '2.17%', '21': '2.09%', '22': '2.09%', '23': '2.09%', '24': '2.09%', '25': '2.09%', '26': '2.09%', '27': '2.09%', '28': '2.09%', '29': '2.02%', '30': '2.02%', '31': '2.02%', '32': '1.94%', '33': '1.94%',

It is difficult to comment on whether our data accurately represents the population without knowing the demographics of the population but for this analysis we will assume the age column has a near uniform distribution. As such we notice that the proportion of most ages in our data is somewhere between 1.6% and 2.2% except for the proportion of 18 and 19 year olds which are both over 5.0%. The ages are skewed which will affect our analysis. A possible remedy for this problem would be to take a random sample of the rows where age is '18' and '19' so that their frequency is around the same as the other ages. However, that is beyond the scope of this project.
We will examine BMI next.

In [82]:
display_numerical(all_data, 'bmi')

          bmi:

     Min = 15.96
     Lower Quartile = 26.29
     Median = 30.4
     Upper Quartile = 34.7
     Max = 53.13
     Range = 37.17
     Interquartile Range = 8.41
     Mean = 30.66
     Standard Deviation = 6.1



BMI is a continuous variable which makes analyzing the data difficult without using visualizations. We could print the frequency and proportion tables but they would be very large and a histogram would be preferred. Judging by the summary statistics, where the median and mean are almost equal and the IQR and standard deviation are relatively small, we can say that BMI appears to follow a normal distribution.
We will examine number of children next.

In [83]:
display_numerical(all_data, 'children')
display_categorical(all_data, 'children')
display_categorical(all_data, 'has_children')

          children:

     Min = 0.0
     Lower Quartile = 0.0
     Median = 1.0
     Upper Quartile = 2.0
     Max = 5.0
     Range = 5.0
     Interquartile Range = 2.0
     Mean = 1.09
     Standard Deviation = 1.21

          children:

     Mode = 0
     Frequency = {'0': 574, '1': 324, '2': 240, '3': 157, '4': 25, '5': 18}
     Proportions = {'0': '42.9%', '1': '24.22%', '2': '17.94%', '3': '11.73%', '4': '1.87%', '5': '1.35%'}

          has_children:

     Mode = yes
     Frequency = {'no': 574, 'yes': 764}
     Proportions = {'no': '42.9%', 'yes': '57.1%'}



We can see that most people have children but the mode of our data is 0 children. Also very few people have more than 3 children. Next we will examine sex.

In [84]:
display_categorical(all_data, 'sex')

          sex:

     Mode = male
     Frequency = {'female': 662, 'male': 676}
     Proportions = {'female': '49.48%', 'male': '50.52%'}



Our data has more males than females but both proportions are nearly 50%. This is a fairly uniform distribution of a discrete variable and we can assume it is accurately represents the population. We will examine smoking status next.

In [85]:
display_categorical(all_data, 'smoker')

          smoker:

     Mode = no
     Frequency = {'no': 1064, 'yes': 274}
     Proportions = {'no': '79.52%', 'yes': '20.48%'}



Most people in our data are not smokers, while smokers are approximately 20%. We would not expect this distribution to be uniform. Without any other data we have to assume this accurately represents our population. Next we will examine region.

In [86]:
display_categorical(all_data, 'region')

          region:

     Mode = southeast
     Frequency = {'northeast': 324, 'northwest': 325, 'southeast': 364, 'southwest': 325}
     Proportions = {'northeast': '24.22%', 'northwest': '24.29%', 'southeast': '27.2%', 'southwest': '24.29%'}



Our data is fairly uniform when it comes to region with all proportions nearly 25%. We have slightly more people from the southeast than any other region.

Now we will analyze how different variables are related to the charges variable. We will look at children, sex, smoker, and region. Without visualizations it is too difficult to examine how age and bmi relate to the charges variable.

In [87]:
display_group_stats(all_data, 'has_children', 'charges')

HAS_CHILDREN

     -no-

          charges:

     Min = 1121.87
     Lower Quartile = 2731.91
     Median = 9856.95
     Upper Quartile = 14449.85
     Max = 63770.43
     Range = 62648.56
     Interquartile Range = 11717.94
     Mean = 12365.98
     Standard Deviation = 12023.29

     -yes-

          charges:

     Min = 1711.03
     Lower Quartile = 5782.76
     Median = 9223.83
     Upper Quartile = 18241.33
     Max = 60021.4
     Range = 58310.37
     Interquartile Range = 12458.57
     Mean = 13949.94
     Standard Deviation = 12138.31



Some things to notice are the upper and lower quartiles and mean are higher for people with children but the median is lower. The other statistcs are not much different.

In [88]:
display_group_stats(all_data, 'children', 'charges')

CHILDREN

     -0-

          charges:

     Min = 1121.87
     Lower Quartile = 2731.91
     Median = 9856.95
     Upper Quartile = 14449.85
     Max = 63770.43
     Range = 62648.56
     Interquartile Range = 11717.94
     Mean = 12365.98
     Standard Deviation = 12023.29

     -1-

          charges:

     Min = 1711.03
     Lower Quartile = 4787.63
     Median = 8483.87
     Upper Quartile = 15905.0
     Max = 58571.07
     Range = 56860.04
     Interquartile Range = 11117.37
     Mean = 12731.17
     Standard Deviation = 11823.63

     -2-

          charges:

     Min = 2304.0
     Lower Quartile = 6257.93
     Median = 9264.98
     Upper Quartile = 20580.88
     Max = 49577.66
     Range = 47273.66
     Interquartile Range = 14322.95
     Mean = 15073.56
     Standard Deviation = 12891.37

     -3-

          charges:

     Min = 3443.06
     Lower Quartile = 6602.14
     Median = 10600.55
     Upper Quartile = 19280.97
     Max = 60021.4
     Range = 56578.34
     Interquartil

The mean increases as you have more children until you have 3 children and then decreases with 4 and 5 children. We have smaller sample size for people with 4 or 5 children and most likely we have fewer smokers which would lower our mean charges.

In [89]:
display_group_stats(all_data, 'sex', 'charges')

SEX

     -female-

          charges:

     Min = 1607.51
     Lower Quartile = 4883.87
     Median = 9412.96
     Upper Quartile = 14455.64
     Max = 63770.43
     Range = 62162.92
     Interquartile Range = 9571.77
     Mean = 12569.58
     Standard Deviation = 11128.7

     -male-

          charges:

     Min = 1121.87
     Lower Quartile = 4591.51
     Median = 9369.62
     Upper Quartile = 19006.69
     Max = 62592.87
     Range = 61471.0
     Interquartile Range = 14415.18
     Mean = 13956.75
     Standard Deviation = 12971.03



Males have higher mean charges and a much higher upper quartile and IQR. The most likely cause is a higher proportion of males are smokers.

In [90]:
display_group_stats(all_data, 'smoker', 'charges')

SMOKER

     -no-

          charges:

     Min = 1121.87
     Lower Quartile = 3984.95
     Median = 7345.41
     Upper Quartile = 11363.02
     Max = 36910.61
     Range = 35788.74
     Interquartile Range = 7378.07
     Mean = 8434.27
     Standard Deviation = 5993.78

     -yes-

          charges:

     Min = 12829.46
     Lower Quartile = 20773.63
     Median = 34456.35
     Upper Quartile = 41034.22
     Max = 63770.43
     Range = 50940.97
     Interquartile Range = 20260.59
     Mean = 32050.23
     Standard Deviation = 11541.55



All the statistics are much higher for smokers than non-smokers. The mean is nearly 4 times as high. We can see that smokers skew the charges to be much higher than they should be which explains the large difference between the mean (13270.42) and the median (9382.03) for charges regardless of smoking status.

In [91]:
display_group_stats(all_data, 'region', 'charges')

REGION

     -northeast-

          charges:

     Min = 1694.8
     Lower Quartile = 5179.07
     Median = 10057.65
     Upper Quartile = 16717.01
     Max = 58571.07
     Range = 56876.27
     Interquartile Range = 11537.94
     Mean = 13406.38
     Standard Deviation = 11255.8

     -northwest-

          charges:

     Min = 1621.34
     Lower Quartile = 4668.91
     Median = 8965.8
     Upper Quartile = 14865.75
     Max = 60021.4
     Range = 58400.06
     Interquartile Range = 10196.84
     Mean = 12417.58
     Standard Deviation = 11072.28

     -southeast-

          charges:

     Min = 1121.87
     Lower Quartile = 4432.31
     Median = 9294.13
     Upper Quartile = 19530.61
     Max = 63770.43
     Range = 62648.56
     Interquartile Range = 15098.3
     Mean = 14735.41
     Standard Deviation = 13971.1

     -southwest-

          charges:

     Min = 1241.57
     Lower Quartile = 4748.71
     Median = 8798.59
     Upper Quartile = 13466.69
     Max = 52590.83
     Range =

Nothing really jumps out as interesting in these statistics except the upper quartile, and thus the IQR, of the southeast. The most likely suspect is that the southeast has a higher proportion smokers than the other regions.
Finally we make a function to test our hypothesis that there is a higher proportion of smokers in the groups with 3 or less children, men, and people from the southeast.

In [92]:
def group1_count2(data, group1, group2):
    new_dict = group_by_column(data, group1)
    dict_count = {}
    print('{} - {} : count'.format(group1.upper(), group2.upper()))
    print()
    for key in new_dict:
        dict2 = group_by_column(new_dict[key], group2)
        for key2 in dict2:
            print(' {} - {} : {}'.format(key, key2, len(dict2[key2])))
            
group1_count2(all_data, 'smoker', 'children')
print()
group1_count2(all_data, 'smoker', 'sex')
print()
group1_count2(all_data, 'smoker', 'region')

SMOKER - CHILDREN : count

 no - 0 : 459
 no - 1 : 263
 no - 2 : 185
 no - 3 : 118
 no - 4 : 22
 no - 5 : 17
 yes - 0 : 115
 yes - 1 : 61
 yes - 2 : 55
 yes - 3 : 39
 yes - 4 : 3
 yes - 5 : 1

SMOKER - SEX : count

 no - female : 547
 no - male : 517
 yes - female : 115
 yes - male : 159

SMOKER - REGION : count

 no - northeast : 257
 no - northwest : 267
 no - southeast : 273
 no - southwest : 267
 yes - northeast : 67
 yes - northwest : 58
 yes - southeast : 91
 yes - southwest : 58


As we can see from these tables our hypothesis is correct. Higher charges are related to higher proportions of smokers in each category.